## Modeling

In [1]:
%set_env TF_USE_LEGACY_KERAS=True

env: TF_USE_LEGACY_KERAS=True


In [2]:
import pandas as pd
import tf_keras as keras 

2025-07-20 01:54:43.045119: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-20 01:54:43.052562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752990883.060887 1988837 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752990883.063426 1988837 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752990883.069750 1988837 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
reviews = pd.read_parquet("../data/2023/Electronics_min.parquet")
items = pd.read_parquet("../data/2023/meta_Electronics.parquet")

In [4]:
reviews

,rating,parent_asin,user_id,timestamp
0,3,B083NRGZMM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-07-18 22:58:37.948
1,1,B07N69T6TM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-06-20 18:42:29.731
2,5,B01G8JO5F2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2018-04-07 09:23:37.534
3,5,B001OC5JKY,AGGZ357AO26RQZVRLGU4D4N52DZQ,2010-11-20 18:41:35.000
4,5,B07CJYMRWM,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,2023-02-17 02:39:41.238
...,...,...,...,...
9999995,4,B0BR6FBPD9,AHG2SVUXNKVWEHVTVRZHIDRCWFQA,2021-07-07 17:10:50.418
9999996,5,B07DVRGM9M,AHG2SVUXNKVWEHVTVRZHIDRCWFQA,2018-07-19 14:04:49.157
9999997,5,B0B3TJMN8P,AGT6T3SMU577AW4KZOWCX7VIY3IQ,2022-08-26 16:29:27.240
9999998,5,B0B3158RQ2,AFY2KJ5YAVB77AAACLXEAHT4CHWQ,2022-12-06 20:21:24.745


In [5]:
items

,title,average_rating,rating_number,description,price,images,parent_asin
0,FS-1051 FATSHARK TELEPORTER V3 HEADSET,3.5,6,[Teleporter V3 The “Teleporter V3” kit sets a ...,None,"[{'hi_res': None, 'large': 'https://m.media-am...",B00MCW7G9M
1,Ce-H22B12-S1 4Kx2K Hdmi 4Port,5.0,1,[HDMI In - HDMI Out],None,[{'hi_res': 'https://m.media-amazon.com/images...,B00YT6XQSE
2,Digi-Tatoo Decal Skin Compatible With MacBook ...,4.5,246,[],19.99,[{'hi_res': 'https://m.media-amazon.com/images...,B07SM135LS
3,NotoCity Compatible with Vivoactive 4 band 22m...,4.5,233,[],9.99,[{'hi_res': 'https://m.media-amazon.com/images...,B089CNGZCW
4,Motorola Droid X Essentials Combo Pack,3.8,64,[all Genuine High Quality Motorola Made Access...,14.99,"[{'hi_res': None, 'large': 'https://m.media-am...",B004E2Z88O
...,...,...,...,...,...,...,...
1610007,"Wintec FileMate Pro USB Flash Drive, 3FMUSB32G...",5.0,1,[--New in retail packaging --Fast USB 2.0 data...,None,[{'hi_res': 'https://m.media-amazon.com/images...,B003NUIU9M
1610008,Tsugar Noise Reduction Wireless Headphones Blu...,1.0,2,[Description: 100% brand new high quality 1.Hi...,None,"[{'hi_res': None, 'large': 'https://m.media-am...",B0BHVY33TL
1610009,"Hardshell Case for MacBook Pro (16-inch, 2021)...",4.6,11,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B09SQGRFFH
1610010,"FYY 12-13.3"" Laptop Sleeve Case Bag, PU Leathe...",4.0,35,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B091JWCSG5


We need to build a compact user-centric representation of preference, collapse review data into a sparse matrix of user -> item preferences. There are some heuristics that need to be applied in the process: 
1. users with few interactions are a very weak signal -- without associations with multiple products, we are not teaching the model about positive associations
2. products with few interactions are also a very weak signal -- we are looking to connect users and items that have tiny interaction graphs are not going to improve our macro-level predictions

In [6]:
# TODO: This should be a configuration option hyper parameter, we can relax this if training isn't suepr computationally expensive
min_ratings = 1000
items = items[items.rating_number > min_ratings]
len(items) 

42604

In [7]:
# The above item filtering serves to reduce the computational complexity as well as 
# reduce sparsity, before we filter reviews make sure we remove those associated with 
# dropped items
all_items = set(items.parent_asin)
reviews = reviews[reviews.parent_asin.isin(all_items)]
reviews.rename(columns={'parent_asin':'item_id'}, inplace=True)

/tmp/ipykernel_1988837/1979523271.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews.rename(columns={'parent_asin':'item_id'}, inplace=True)


In [8]:
users = reviews.groupby(['user_id']).rating.count()
users = pd.DataFrame(users).reset_index()
users.rename(columns={'rating':'ratings'}, inplace=True)

In [9]:
# This is a configuration parameter, as above ... relax if we don't have issues with compute
min_reviews = 95
users = users[users.ratings > min_reviews] 
users

,user_id,ratings
23050,AE3LZ5EMSN5BJ5USW4SJRDHI5M4A,122
24389,AE3ORZRCEZPHFNQV5AT27AZXRCBA,116
31444,AE45ZGMA3H5BVWXV5ZOUTXMWDKXQ,132
41304,AE4SX2IMZGVQSS5SFWP6NEE2HQ4Q,96
42562,AE4VNCNZ5XJON3GQWHFJPZ626HVQ,101
...,...,...
1693773,AHMNA5UK3V66O2V3DZSBJA4FYMOA,233
1792892,AHTDYGXHONM2BHENDRMKMC34ZZZA,96
1850059,AHX7ER6ODLRXMZF3WLEFJVRUJUXQ,122
1862291,AHXZR7HLPSKRUPHX35GKRLVTX3ZA,122


❗in the notebook, ratings are thresholded ... do we need to follow suit? what are the ramifications if we don't? OH... in the notebook, a click is an interaction, there's no middle ground. the network is going to operate on 0s or 1s. by leaving low reviews in our matrix, the network would learn to recommend things users have interacted with, but not necessarily positively. our case is the same, a review is an interaction. we're aiming to recommend, and we should not want to recommend low reviews. so filter... 

In [10]:
#TODO: decide if we need to keep the low reviews around 
reviews = reviews[reviews.rating >= 3]

In [11]:
# Discard reviews by users outside our core group 
reviews = reviews[reviews.user_id.isin(set(users.user_id))]

In [12]:
reviews

,rating,item_id,user_id,timestamp
487,5,B07H65KP63,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2023-01-09 18:29:43.584
489,5,B0BC87S9SY,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2023-01-01 00:38:53.148
490,5,B09B36FJVT,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2023-01-01 00:28:42.952
491,5,B095KD3MNN,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-11-26 21:08:54.526
495,5,B07Z18JVPH,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-08-01 19:04:43.671
...,...,...,...,...
7568118,4,B004QGXWSQ,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-03-09 15:16:13.000
7568119,5,B07961C64Q,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-03-09 15:10:31.000
7568121,5,B009RWAYTE,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-02-12 15:21:44.000
7568122,5,B07BSJCMGS,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-02-12 15:14:18.000


In [13]:
matrix_size = len(users) * len(items)
matrix_size

3067488

In [14]:
from recommenders.datasets.sparse import AffinityMatrix

In [15]:
import numpy as np 
import pandas as pd 

In [16]:
reviews.columns

Index(['rating', 'item_id', 'user_id', 'timestamp'], dtype='object')

In [17]:
reviews

,rating,item_id,user_id,timestamp
487,5,B07H65KP63,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2023-01-09 18:29:43.584
489,5,B0BC87S9SY,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2023-01-01 00:38:53.148
490,5,B09B36FJVT,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2023-01-01 00:28:42.952
491,5,B095KD3MNN,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-11-26 21:08:54.526
495,5,B07Z18JVPH,AFZUK3MTBIBEDQOPAK3OATUOUKLA,2022-08-01 19:04:43.671
...,...,...,...,...
7568118,4,B004QGXWSQ,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-03-09 15:16:13.000
7568119,5,B07961C64Q,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-03-09 15:10:31.000
7568121,5,B009RWAYTE,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-02-12 15:21:44.000
7568122,5,B07BSJCMGS,AEOGM6IKVQXQUUMQSLR2DCLDFFUA,2013-02-12 15:14:18.000


In [18]:
# NOTE: Strategy adapted from tutorials available in the Recommenders project, see 
# https://github.com/recommenders-team/recommenders/tree/main
from recommenders.datasets.python_splitters import python_random_split

# Split along user boundaries to ensure no leakage of preference between train and test
train_users, test_users, val_users = python_random_split(users, [.9, .05, .05])

/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [19]:
print(train_users.shape, test_users.shape, val_users.shape)

(65, 2) (3, 2) (4, 2)


In [20]:
train = reviews[reviews.user_id.isin(train_users.user_id)]
val = reviews[reviews.user_id.isin(val_users.user_id)]
test = reviews[reviews.user_id.isin(test_users.user_id)]

In [21]:
print(train.shape, val.shape, test.shape)

(7091, 4) (518, 4) (320, 4)


In [22]:
# Technique from Recommenders (see https://github.com/recommenders-team/recommenders/blob/45e1b215a35e69b92390e16eb818d4528d0a33a2/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) 
# to improve utility of validation set during training - only allow items in
# the validation set that are also present in the train set
val = val[val.item_id.isin(train.item_id.unique())]

In [23]:
val.shape

(309, 4)

In [24]:
from recommenders.datasets.python_splitters import python_stratified_split 

# Another technique employed in Recommenders (see above link for notebook), for in-flight validation to be 
# meaningful during training, our validation set needs not just ground truth, but unseen validation samples 
# to see if predictions for validation users are relevant (to those users). Anyway, break down our val and test 
# sets again to support this strategy
val_src, val_target = python_stratified_split(
    data=val, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )
test_src, test_target = python_stratified_split(
    data=test, 
    ratio=0.8, 
    filter_by="item", 
    col_user="user_id", 
    col_item="item_id"
    )

/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/recommenders/datasets/python_splitters.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["random"] = np.random.rand(data.shape[0])


In [25]:
print(val.shape, " -> ", val_src.shape, val_target.shape)
print(test.shape, " -> ", test_src.shape, test_target.shape)

(309, 5)  ->  (306, 4) (3, 4)
(320, 5)  ->  (319, 4) (1, 4)


In [26]:
#to use standard names across the analysis 
header = {
        "col_user": "user_id",
        "col_item": "item_id",
        "col_rating": "rating",
        # Unclear why this doesn't also eat a timestamp, but many of the functions that split temporally use, fortunately 
        # the column 'timestamp' (i.e. DEFAULT_TIMESTAMP_COL='timestamp') so I think we're fine. 
        # "col_timestamp" : "timestamp"
    }

train_matrix = AffinityMatrix(df=train, **header)
val_matrix = AffinityMatrix(df=val, **header)
val_src_matrix = AffinityMatrix(df=val_src, **header)
val_tgt_matrix = AffinityMatrix(df=val_target, **header)
test_src_matrix = AffinityMatrix(df=test_src, **header)
test_tgt_matrix = AffinityMatrix(df=test_target, **header)

In [27]:
# This generates a sparse array of user vectors, aka user-item matrix
# X[0] is the first user in the list, with entries for all items known when the matrix was constructed in that row
train, _, _ = train_matrix.gen_affinity_matrix()
val, _, _ = val_matrix.gen_affinity_matrix() 
val_src, _, _ = val_src_matrix.gen_affinity_matrix()
val_tgt, _, _ = val_tgt_matrix.gen_affinity_matrix()
test_src, _, _ = test_src_matrix.gen_affinity_matrix()
test_tgt, _, _ = test_src_matrix.gen_affinity_matrix()

In [28]:
from recommenders.utils.python_utils import binarize

train = binarize(train, 3)
val = binarize(train, 3)
val_src = binarize(val_src, 3) 
val_tgt = binarize(val_tgt, 3)
test_src = binarize(test_src, 3)
test_tgt = binarize(test_tgt, 3)

In [29]:
# TODO: Make sure this is reported during training/configuration
sparsity = np.count_nonzero(train)/(train.shape[0]*train.shape[1])
sparsity

0.021021061273729536

## Model Design 

In [30]:
keras.__version__ 

'2.19.0'

In [31]:
from recommenders.models.vae.standard_vae import StandardVAE

In [32]:
model = StandardVAE(
    n_users = train.shape[0], 
    original_dim = train.shape[1],
    intermediate_dim=250, 
    latent_dim=50, 
    n_epochs=1, 
    batch_size=1, 
    k=10, 
    verbose=1, 
    seed=4, 
    save_path="models/svae.hdf5", 
    drop_encoder=0.5, 
    drop_decoder=0.5, 
    annealing=False, 
    beta=1.0) 

I0000 00:00:1752990895.769779 1988837 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16874 MB memory:  -> device: 0, name: NVIDIA RTX A5500, pci bus id: 0000:01:00.0, compute capability: 8.6


In [34]:
model.fit(
    x_train=train, 
    x_valid=val, 
    x_val_tr=val_src, 
    x_val_te=val_tgt, 
    mapper=val_matrix,
    )

/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/recommenders/models/vae/standard_vae.py:402: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  self.model.fit_generator(


TypeError: in user code:

    File "/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1381, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1370, in run_step  **
        outputs = model.train_step(data)
    File "/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1148, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/grim/projects/school/deepcart/venv/lib/python3.12/site-packages/tf_keras/src/engine/training.py", line 1206, in compute_loss
        return self.compiled_loss(
    Fi

Managing text-based reviews at this scale could be a challenge, and I'd like to steer clear of LLMs for this effort. We could do an embedding on the review and use that for similarity, but we have pretty rich item data. Perhaps let's ignore the collaborative aspect here and build a shopping interface that: 
- surfaces the most popular items, and encourages you to add items to your shopping cart for a big discount/promo
- based on clicks and cart items, improves the recommendations and surfaces new products

We can use an autoencoder to accept a sparse matrix of users and items, learn to reproduce that matrix, and in so doing support prediction on missing values. However, this matrix is of size users x items, which here is 1.8e7 x 1.6e6 = 28,125,000,000 KB (best-case, higher if stored as np floats) ~= 26 TB !! WTF. 
- In the standard VAE example (https://github.com/recommenders-team/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb) the clicks are turned into a histogram for each user ... so we have n_user vector of length n_items... then I guess each of these is a training sample. The VAE presumably learns, given a sparse user vector, to predict every rating. This takes the complexity down and gives us a training set we can iterate over. 

Let's avoid any distributional pressure (present in VAE, SVAE, disentangled VAE) and go for a basic autoencoder using the strategy laid out above, i.e.

## Autoencoder Prototype

In [ ]:
import torch 
import pandas as pd 
import torch
from tqdm import tqdm
import os
import math 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import wandb

In [ ]:
class Autoencoder(nn.Module):
    """
    Autoencoder

    NOTE: with cues from https://www.geeksforgeeks.org/deep-learning/implementing-an-autoencoder-in-pytorch/
    """

    def __init__(self, dims=1000):
        """
        Initialize a new object 
        """
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(dims, 500),
            nn.Linear(500, 75),
        )
        self.decoder = nn.Sequential(
            nn.Linear(75, 500),
            nn.Linear(500, dims),
        )

    def forward(self, x):
        """
        Implement our forward pass 
        """
        h = self.encoder(x) 
        r = self.decoder(h)

        return r

In [ ]:
class DeepCartDataset(torch.utils.data.Dataset): 
    """
    Custom pytorch-compatible dataset. Adapted from 
    https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#creating-a-custom-dataset-for-your-files
    """
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None): 

        self.img_labels = pd.read_csv(annotations_file)

        #TODO: implement

    def __len__(self): 
        return len(self.img_labels) 
    
    def __getitem__(self, idx): 
        #TODO: implement
        pass

In [ ]:
def get_data_loader(batch_size=5, shuffle=True): 
    """
    Retrieve a pytorch-style dataloader 
    """

    #TODO: implement
    #transform = transforms.Compose([
    #     transforms.ConvertImageDtype(torch.float),
    #     transforms.Normalize(mean=[0.5], std=[0.5])
    #])

    #data = DeepCartDataset(transform=transform)
    #loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=shuffle)
    
    #return loader
    pass

In [ ]:
def train(loader, model, loss_interval=20, epochs=2, lr=0.01, momentum=0.9):
    """
    Train the model with the provided dataset

    NOTE: this is a similar training loop as we used for our vision model in the 
    the vision project, forward pass
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    train_loss = []

    tqdm.write(f"Starting training run...")    
    # TODO: configure WandB
    # see https://docs.wandb.ai/guides/integrations/pytorch/
    config = {}
    run = wandb.init(config=config) 

    model.train()
    model = model.to(device)
    
    loss_fn = nn.CrossEntropyLoss()

    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(loader):

            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)

            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            # collect metrics
            running_loss += loss.item()

            if (i % loss_interval) == (loss_interval - 1): 
                train_loss.append(running_loss / loss_interval)
                tqdm.write(f"[{epoch + 1}, {i + 1:5d}] loss: {running_loss / loss_interval:.3f}")
                running_loss = 0 
    
    tqdm.write("Training complete!") 

    return train_loss 